<a href="https://colab.research.google.com/github/Computer-CGuy/Submerge/blob/master/SUBMERGE_Pytorch_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUBMERGE NETWORK



#### Including modules

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import cv2
from google.colab.patches import cv2_imshow

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

### Seperate Functions



*   ``` getBinaryImage(img) ``` converts image to binary values(0 and 1)



*    ``` merge(img1,img2) ``` AND 2 images based on weigths

*    ``` fading(img3,cord) ``` takes an image and add cord, and make an epicenter on the image at the cord






In [0]:
def getBinaryImage(img):
  ret = np.where(img<0.5,0,1)
  return ret

In [0]:
def getBinaryImagePt(img):
  a1 = nn.Threshold(0.5,1)
  # img*=-1
  img = a1(img)
  # print(img)
  img = torch.mul(img,-1)
  # print(img)
  a2 = nn.Threshold(-0.99,0)
  img = a2(img)
  
  # img*=-1
  img = torch.mul(img,-1)

  # img = nn.Threshold(0.5,1)
  # ones = torch.ones(img.shape)
  # zeros = torch.zeros(img.shape)
  # ret = torch.where(img<0.5,zeros,ones)
  return img

In [0]:
def merge(img1,img2):
  ret = np.multiply(img1,img2)
  return ret

In [0]:
def mergePt(img1,img2):
  rel = torch.mul(img1,img2)
  # ret = img1*img2
  return ret

In [0]:
def fading(img3,cord):
  
  for color in range(1,255,1):
    cv2.circle(img3,cord,2+int(400/color),(color),thickness=-1)
  return img3

In [0]:
def drawCircle(color,cord=(12,12)):
  img = np.zeros((24,24))  
  r = 5
  for x in range(24):
    for y in range(24):
      dist = ((((x-cord[0])**2)+((y-cord[1])**2))**(1/2))
      if(dist<r):
        img[x][y] = color
  return img
        # print(r-dist)
  # cv2_imshow(img)

In [0]:
def fadingPt(cords):
  cords = cords.cpu().detach().numpy()
  img = np.zeros((10,24,24))
  for x in range(10):
    # print(cords[x][0])
    for color in range(1,255,1):
      c = torch.Variable(color)
      
      # cv2.circle(img[x],(int(cords[x][0]),int(cords[x][1])),2+int(400/color),(color),thickness=-1)
  return img

In [0]:
def fadingPt(cords):
  cords = cords.cpu().detach().numpy()
  # print(cords)
  img = np.zeros((10,24,24))
  for x in range(10):
    # print(cords[x][0])
    for color in range(1,255,1):
      cv2.circle(img[x],(int(cords[x][0]),int(cords[x][1])),2+int(400/color),(color),thickness=-1)
  img = torch.Tensor(img)
  return img

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<FadingBackward>)


### Truth table of merge

In [0]:
 img1 img2
1    1    1
0    0    0
1    0    0
0    0    0

### ``` fading()``` example

In [0]:
img2 = np.zeros((24,24))
cv2_imshow(fading(img2,(10,10)))

### Submerge 2 images

In [0]:
img = torch.rand((1,1,24,24))

In [0]:
cv2_imshow(getBinaryImage(img[0][0].detach().numpy())*255)

In [0]:
cv2_imshow(merge(getBinaryImage(img[0][0].detach().numpy()),img2))

### Application-Type Implementation


*   Takes in weights (2D Layer), Location(3D (X,Y) at weights indices) layers.
*   Applies the submerge algorithms
*   Sums up all the outputs.
*   Later difference will also be implemented.

#### Sources


*   https://stackoverflow.com/questions/16229823/how-to-multiply-numpy-2d-array-with-numpy-1d-array
*   https://pytorch.org/docs/stable/torch.html#torch.where


In [0]:
weights = torch.Tensor([[1,2,4]])
loc = torch.Tensor([[[0,1],[12,12],[90,90]]])

In [0]:
weights.shape

torch.Size([1, 3])

In [0]:
loc.shape

torch.Size([1, 3, 2])

In [0]:
memory = getBinaryImagePt(torch.rand((1,10,24,24)))

In [0]:
cv2_imshow(memory[0][0].detach().numpy()*255)

In [0]:
cords = (torch.rand((10,2))*24).round().int()

In [0]:
subms = torch.Tensor(fadingPt(cords))
add_subms = torch.mul(subms,memory)

In [0]:
add_subms.shape

torch.Size([1, 10, 24, 24])

In [0]:
added = (add_subms.mean(dim=1))

In [0]:
added.shape

torch.Size([1, 24, 24])

In [0]:
cv2_imshow(added[0].cpu().detach().numpy())

In [0]:
for h in range(10):
  cv2_imshow(add_subms[0][h].cpu().detach().numpy())

In [0]:
weights = torch.rand(10)
mul = add_subms* weights[:,None,None]
mul.shape

torch.Size([1, 10, 24, 24])

In [0]:
image = torch.sum(mul,1)
image.shape
cv2_imshow(image.detach().numpy()[0])

### Functions Equality

In [0]:
def getImageAdd(cords_add,memory_add,weights_add):
  memory = getBinaryImagePt(memory_add)
  # cords = (torch.rand((10,2))*24).round().int()
  subms_add = torch.Tensor(fadingPt(cords_add))
  add_subms = torch.mul(subms_add,memory_add)
  # weights = torch.rand(10)
  # add_subms.mul()
  mul = add_subms * weights_add[:,None,None]
  # u = weights[:,None,None].mul(add_subms)
  image = torch.sum(mul,dim=1)
  # mul.sum(dim=1)
  return image

In [0]:
def getImageSub(cords_sub,memory_sub,weights_sub):
  memory = getBinaryImagePt(memory_sub)
  # cords = (torch.rand((10,2))*24).round().int()
  subms_sub = torch.Tensor(fadingPt(cords_sub))
  add_subms = torch.mul(subms_sub,memory_sub)
  # weights = torch.rand(10)
  # add_subms.mul()
  mul = add_subms * weights_sub[:,None,None]
  # u = weights[:,None,None].mul(add_subms)
  image = torch.sum(mul,dim=1)
  # mul.sum(dim=1)
  return image

In [0]:
weights_add = torch.rand(10,requires_grad=True)
memory_add = getBinaryImagePt(torch.rand((1,10,24,24),requires_grad=True))
cords_add = (torch.rand((10,2),requires_grad=True)*24).round().int()

In [0]:
torch.manual_seed(200)

In [0]:
add = (getImageAdd(cords_add,memory_add,weights_add)[0].cpu().detach().numpy())

cv2_imshow(add)

In [0]:
weights_sub = torch.rand(10)
memory_sub = getBinaryImagePt(torch.rand((1,10,24,24)))
cords_sub = (torch.rand((10,2))*24).round().int()

In [0]:
sub = (getImageSub(cords_sub,memory_sub,weights_sub)[0].cpu().detach().numpy())
cv2_imshow(sub)

In [0]:
cv2_imshow(add-sub)    

### Pytorch Implementation

In [0]:
class Fading(torch.autograd.Function):
    @staticmethod
    def forward(ctx,input,mines,memory_add,memory_sub):
        indices = mines*3
        add = input[:indices]
        
        cords_add = add[:(mines*2)]
        cords_add = cords_add.view((10,2)).int()
        # print(cords_add)
        weights_add = add[(mines*2):]
        add1 = getImageAdd(cords_add,memory_add,weights_add)

        sub = input[indices:]
        cords_sub = sub[:(mines*2)].int()
        cords_sub = cords_sub.view((10,2)).int()

        weights_sub = sub[(mines*2):]
        sub1 = getImageSub(cords_sub,memory_sub,weights_sub)
        output = add1-sub1
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        # print(ctx)
        # print(input)
        ctx.save_for_backward(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        # print(ctx.saved_tensors)
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        # grad_input[input < 0] = 0
        return grad_input

In [0]:
weights_add = torch.rand(10,requires_grad=True)
memory_add = getBinaryImagePt(torch.rand((1,10,24,24),requires_grad=True))
cords_add = (torch.rand((20),requires_grad=True)*24).round()

In [0]:
weights_sub = torch.rand(10)
memory_sub = getBinaryImagePt(torch.rand((1,10,24,24)))
cords_sub = (torch.rand((20),requires_grad=True)*24).round()


In [0]:
ins = torch.cat((cords_add,weights_add,cords_sub,weights_sub))
mines = 10

In [0]:
output = Fading.apply(ins,10,memory_add,memory_sub)

In [17]:
cv2_imshow(output.detach().cpu().numpy()[0])


### SUBMERGE on Convolution blocks

In [0]:
total_filters = 5
conv1 = torch.nn.Conv2d(1,total_filters, 5)
filters = conv1(img
for filter_no in range(total_filters):
  # cv2_imshow(filters.detach().numpy()[0][filter_no]*255)
  # cv2_imshow(merge(getBinaryImage((merge(getBinaryImage(filters[0][filter_no].detach().numpy()),img2))),img2))
  img1 = getBinaryImage(filters[0][filter_no].detach().numpy())

  img2 = np.zeros((24,24))
  fading(img2,(10,10))

  cv2_imshow(merge(img1,img2))

In [0]:
img = torch.rand((1,1,28,28))
  
img2 = np.zeros((24,24))
fading(img2,(20,20))

cv2_imshow(merge(img1,img2))

ValueError: ignored

In [0]:
filters.detach().numpy()[0][0]

In [0]:
filters.detach().numpy()[0][29]*img

In [0]:
num_epochs = 5
batch_size = 100
learning_rate = 0.001
noises = int(42000/6)

train_data = pd.read_csv('train.csv')
train_data.head()

# test_data = pd.read_csv('./data/test.csv')
# test_data.head()

class CustomedDataSet(torch.utils.data.Dataset):
    def __init__(self, train=True):
        self.train = train
        if self.train :
            dataset = pd.read_csv('train.csv')
            # print(dataset)
            trainY = dataset.iloc[:,0].values
            trainX = dataset.iloc[:,1:].values.reshape(42000 ,1,28,28)
            
            print(trainY)
            
            
            # print(np.argmax(trainY))
            
            self.datalist = trainX
            self.labellist = trainY
        else:
            testX = pd.read_csv('./data/test.csv')
            testX = testX.as_matrix().reshape(testX.shape[0], 1, 28, 28)
            self.datalist = testX
            
    def __getitem__(self, index):
        if self.train:
            return torch.Tensor(self.datalist[index].astype(float)),self.labellist[index]
        else:
            return torch.Tensor(self.datalist[index].astype(float))
    
    def __len__(self):
        return self.datalist.shape[0]

train_dataset = CustomedDataSet()

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=2)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10,100)
        self.fc2 = nn.Linear(100,200) 
        self.fc3 = nn.Linear(200,300) 
        self.fc4 = nn.Linear(300,500) 
        self.fc5 = nn.Linear(500,28*28)
        self.conv1 = torch.nn.Conv2d(1,30, 5)
        self.conv4 = torch.nn.ConvTranspose2d(30,1,5)
        
        self.soft = nn.Softmax2d()
        #self.fc5 = nn.Linear(28*28,)
    def forward(self, x, sig = 0):
        x = nn.functional.leaky_relu(self.fc1(x))
        # x = self.drop(x)
        x = nn.functional.leaky_relu(self.fc2(x))
        # x = self.drop(x)
        x = nn.functional.leaky_relu(self.fc3(x))
        # x = self.drop(x)
        x = nn.functional.leaky_relu(self.fc4(x))
        # x = self.drop(x)
        x = nn.functional.relu(self.fc5(x))
        
        
        x = torch.reshape(x,(1,1,28,28))
        
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv4(x))

        if(sig==1):
          x = nn.functional.sigmoid(x)
        if(sig==2):
          x = self.soft(x)
        #x = self.fc5(x)
        return x